In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json
import sys
import math 

In [3]:
import sys
sys.path.append('/usr0/home/naveenr/projects/patient_provider')

In [4]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import *
from patient.lp_policies import *
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources, one_shot_policy, MyEncoder

In [5]:
is_jupyter = 'ipykernel' in sys.modules

In [36]:
if is_jupyter: 
    seed        = 43
    num_patients = 50
    num_providers = 50
    provider_capacity = 1
    noise = 0.1
    fairness_constraint = -1
    num_trials = 25
    utility_function = "normal"
    order="uniform"
    online_arrival = False   
    new_provider = False 
    out_folder = "dynamic"
    average_distance = 20.2
    max_shown = 25
    online_scale = 1
    verbose = True 
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--max_shown',        help='How many providers to show at most', type=int, default=25)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=1)
    parser.add_argument('--noise', help='Noise in theta', type=float, default=0.1)
    parser.add_argument('--average_distance', help='Maximum distance patients are willing to go', type=float, default=20.2)
    parser.add_argument('--fairness_constraint', help='Maximum difference in average utility between groups', type=float, default=-1)
    parser.add_argument('--num_trials', help='Number of trials', type=int, default=100)
    parser.add_argument('--utility_function', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument('--online_scale', help='Which folder to write results to', type=float, default=1)
    parser.add_argument('--order', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument("--online_arrival",action="store_true",help="Patients arrive one-by-one")
    parser.add_argument("--new_provider",action="store_true",help="Are we simulating a new provider matching")
    parser.add_argument("--verbose",action="store_true",help="Are we storing all the information")
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    num_trials = args.num_trials
    noise = args.noise
    average_distance = args.average_distance
    fairness_constraint = args.fairness_constraint
    provider_capacity = args.provider_capacity
    max_shown = args.max_shown
    utility_function = args.utility_function
    order = args.order
    online_arrival = args.online_arrival
    new_provider = args.new_provider 
    online_scale = args.online_scale
    out_folder = args.out_folder
    verbose = args.verbose 
    
assert not(online_arrival and new_provider)
save_name = secrets.token_hex(4)  

In [37]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'max_shown': max_shown,
        'utility_function': utility_function, 
        'order': order, 
        'num_trials': num_trials, 
        'noise': noise, 
        'average_distance': average_distance,
        'online_arrival': online_arrival,
        'new_provider': new_provider,
        'fairness_constraint': fairness_constraint,
        'online_scale': online_scale, 
        'verbose': verbose} 

## Baselines

In [38]:
seed_list = [seed]
restrict_resources()

In [39]:
if not online_arrival and fairness_constraint == -1:
    policy = one_shot_policy 
    per_epoch_function = offer_everything
    name = "offer_everything"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['offer_everything_num_matches'])/num_patients,np.mean(results['offer_everything_patient_utilities'])))

offer_everything policy
Matches 0.49520000000000003, Utilities 0.8195294635802793


In [10]:
if not online_arrival and fairness_constraint == -1:
    policy = one_shot_policy
    per_epoch_function = greedy_policy
    name = "greedy"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

In [25]:
results.keys()

dict_keys(['parameters', 'offer_everything_patient_utilities', 'offer_everything_chosen_providers', 'offer_everything_num_matches', 'offer_everything_assortments', 'offer_everything_patient_orders', 'offer_everything_new_providers', 'offer_everything_options'])

In [11]:
if not online_arrival and fairness_constraint == -1:
    policy = one_shot_policy
    per_epoch_function = optimal_policy
    name = "omniscient_optimal"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function,use_real=True)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

## Optimization-Based

In [12]:
if not online_arrival:
    policy = one_shot_policy
    if fairness_constraint != -1:
        per_epoch_function = lambda p: lp_policy(p,fairness_constraint=fairness_constraint)
    else:
        per_epoch_function = lp_policy
    name = "lp"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

In [13]:
# policy = one_shot_policy
# per_epoch_function = full_milp_policy
# name = "full_milp_policy"
# print("{} policy".format(name))

# rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

# for key in rewards:
#     results['{}_{}'.format(name,key)] = rewards[key]
# print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

In [14]:
# policy = one_shot_policy
# per_epoch_function = full_lp_policy
# name = "full_lp_policy"
# print("{} policy".format(name))

# rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

# for key in rewards:
#     results['{}_{}'.format(name,key)] = rewards[key]
# print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

In [15]:
if fairness_constraint == -1:
    policy = one_shot_policy
    per_epoch_function = greedy_justified
    name = "greedy_justified"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

greedy_justified policy
On trial 0
On trial 1
On trial 2
On trial 3
On trial 4
On trial 5
On trial 6
On trial 7
On trial 8
On trial 9
On trial 10
On trial 11
On trial 12
On trial 13
On trial 14
On trial 15
On trial 16
On trial 17
On trial 18
On trial 19
On trial 20
On trial 21
On trial 22
On trial 23
On trial 24
Matches 0.5714285714285714, Utilities 0.583625960070932


In [18]:
rewards['new_providers']

[[-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1]]

## Save Data

In [ ]:
save_path = get_save_path(out_folder,save_name)

In [ ]:
delete_duplicate_results(out_folder,"",results)

In [ ]:
json.dump(results,open('../../results/'+save_path,'w'),cls=MyEncoder)